In [67]:
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium
from urllib.parse import  urlparse
import time
import lxml
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import re
import pickle
from contextlib import suppress
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate, KFold
from tqdm import tqdm

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from collections import Counter

In [68]:
ds_star = pd.read_csv("C:/netsong7/pythonwork/AI/finalproject/data cleaning(final)/shopping_ds.csv")
ds_star.drop_duplicates(subset=['type', 'review', 'star'], inplace=True)
ds_star["label"] = 0
ds_star.loc[ds_star["star"] > 3.5, "label"] = 1
del ds_star["Unnamed: 0"]
ds_star_1_dummy, ds_star_1_train = train_test_split(ds_star[ds_star["label"]==1], test_size=5432, random_state = 42)
ds_star_0_train = ds_star[ds_star["label"]==0]
ds_star_balance = pd.concat([ds_star_1_train, ds_star_0_train])
train_data, test_data = train_test_split(ds_star_balance, test_size=0.3, random_state = 0)
train_data.drop_duplicates(subset=["review", "star"], inplace=True)
test_data.drop_duplicates(subset=["review", "star"], inplace=True)
train_data.loc[train_data.review.isnull()]
train_data = train_data.dropna(how = 'any')
train_data['review'] = train_data['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data.drop_duplicates(subset = ['review'], inplace=True) # 중복 제거
test_data['review'] = test_data['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['review'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
train_data['review'] = train_data['review'].str.replace('^ +', "")
train_data['review'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

type      0
review    7
star      0
label     0
dtype: int64


C:\Users\not14\anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\not14\anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\not14\anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\not14\anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWa

In [69]:
okt = Okt()
#stopwords = ['가','게','고','과','네','는','다','도','들','듯','를','에','와','으로',
#             '은','을','의','이','인','임','자','잘','좀','지','하다','한']
stopwords = pd.read_table("C:/netsong7/pythonwork/AI/stopwords1.txt", encoding='utf-8-sig', header=None,error_bad_lines=False)
stopwords = list(stopwords[0])

X_train = []
for sentence in train_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

X_test = []
for sentence in test_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
vocab_size = total_cnt - rare_cnt + 1    

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

max_len = 100



b'Skipping line 276: expected 1 fields, saw 2\n'
C:\Users\not14\anaconda3\envs\tf1\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [70]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# LSTM model

In [71]:
from keras.layers import *
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint

CLASS_COUNT = 10

base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg',
)
base_model.trainable = False

model = Sequential([
  base_model,
  Dense(CLASS_COUNT, activation='softmax'),
])

In [73]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, 100))
model_lstm.add(LSTM(128))
model_lstm.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_lstm.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_lstm = model_lstm.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)


Train on 6020 samples, validate on 1506 samples
Epoch 1/15
6020/6020 [==============================] - 64s 11ms/step - loss: 0.6031 - acc: 0.6934 - val_loss: 2.1129 - val_acc: 0.5212

Epoch 00001: val_acc improved from -inf to 0.52125, saving model to best_model.h5
Epoch 2/15
6020/6020 [==============================] - 61s 10ms/step - loss: 0.4721 - acc: 0.7831 - val_loss: 0.4948 - val_acc: 0.7656

Epoch 00002: val_acc improved from 0.52125 to 0.76560, saving model to best_model.h5
Epoch 3/15
6020/6020 [==============================] - 61s 10ms/step - loss: 0.4207 - acc: 0.8047 - val_loss: 0.4975 - val_acc: 0.7510

Epoch 00003: val_acc did not improve from 0.76560
Epoch 4/15
6020/6020 [==============================] - 60s 10ms/step - loss: 0.3896 - acc: 0.8233 - val_loss: 0.4824 - val_acc: 0.7769

Epoch 00004: val_acc improved from 0.76560 to 0.77689, saving model to best_model.h5
Epoch 5/15
6020/6020 [==============================] - 62s 10ms/step - loss: 0.3608 - acc: 0.8370 - v

In [74]:
from sklearn.metrics import confusion_matrix
y_pred = model_lstm.predict(X_test)
y_pred_ = []

for i in range(len(y_pred)):
    if y_pred[i][0] > 0.5:
        y_pred_.append(1)
    else:
        y_pred_.append(0)
y_pred_

print(confusion_matrix(y_test, y_pred_))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_))

[[1171  468]
 [ 367 1196]]
              precision    recall  f1-score   support

           0       0.76      0.71      0.74      1639
           1       0.72      0.77      0.74      1563

    accuracy                           0.74      3202
   macro avg       0.74      0.74      0.74      3202
weighted avg       0.74      0.74      0.74      3202



# GRU

In [78]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [79]:
from keras.layers import *
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint

CLASS_COUNT = 10

base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg',
)
base_model.trainable = False

model = Sequential([
  base_model,
  Dense(CLASS_COUNT, activation='softmax'),
])

In [80]:
model_GRU = Sequential()
model_GRU.add(Embedding(vocab_size, 100))
model_GRU.add(GRU(128))
model_GRU.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_GRU.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_GRU = model_GRU.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Train on 6020 samples, validate on 1506 samples
Epoch 1/15
6020/6020 [==============================] - 85s 14ms/step - loss: 0.5650 - acc: 0.7121 - val_loss: 0.4860 - val_acc: 0.7669

Epoch 00001: val_acc improved from -inf to 0.76693, saving model to GRU_model.h5
Epoch 2/15
6020/6020 [==============================] - 80s 13ms/step - loss: 0.4496 - acc: 0.7826 - val_loss: 0.5576 - val_acc: 0.7224

Epoch 00002: val_acc did not improve from 0.76693
Epoch 3/15
6020/6020 [==============================] - 78s 13ms/step - loss: 0.4140 - acc: 0.8022 - val_loss: 0.5293 - val_acc: 0.7457

Epoch 00003: val_acc did not improve from 0.76693
Epoch 4/15
6020/6020 [==============================] - 77s 13ms/step - loss: 0.3815 - acc: 0.8231 - val_loss: 0.4903 - val_acc: 0.7576

Epoch 00004: val_acc did not improve from 0.76693
Epoch 5/15
6020/6020 [==============================] - 78s 13ms/step - loss: 0.3575 - acc: 0.8332 - val_loss: 0.5167 - val_acc: 0.7510

Epoch 00005: val_acc did not improve

In [81]:
from sklearn.metrics import confusion_matrix
y_pred = model_GRU.predict(X_test)
y_pred_ = []

for i in range(len(y_pred)):
    if y_pred[i][0] > 0.5:
        y_pred_.append(1)
    else:
        y_pred_.append(0)
y_pred_

print(confusion_matrix(y_test, y_pred_))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_))

[[1069  570]
 [ 283 1280]]
              precision    recall  f1-score   support

           0       0.79      0.65      0.71      1639
           1       0.69      0.82      0.75      1563

    accuracy                           0.73      3202
   macro avg       0.74      0.74      0.73      3202
weighted avg       0.74      0.73      0.73      3202



In [90]:
from keras.models import load_model

model_GRU.save('model_GRU.h5')

# RNN

In [82]:
from keras.layers import SimpleRNN
import keras
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

In [83]:
from keras.utils import to_categorical
X_train_rnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
print(X_train_rnn.shape)

X_test_rnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(X_test_rnn.shape)

# y_data = np.concatenate((y_train, y_test))
# y_data = to_categorical(y_data)
# y_data

# y_train = y_data[:10750]
# y_test = y_data[10750:]

print(y_train.shape)
print(y_test.shape)

(7526, 100, 1)
(3202, 100, 1)
(7526,)
(3202,)


In [84]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(units=50, input_shape=(100, 1), return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [85]:
model_rnn = KerasClassifier(build_fn=vanilla_rnn, epochs=20, batch_size=50, verbose=1)
model_rnn.fit(X_train_rnn, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Train on 6020 samples, validate on 1506 samples
Epoch 1/15
6020/6020 [==============================] - 8s 1ms/step - loss: 0.6946 - acc: 0.5229 - val_loss: 0.6845 - val_acc: 0.5784

Epoch 00001: val_acc did not improve from 0.76693
Epoch 2/15
6020/6020 [==============================] - 3s 537us/step - loss: 0.6836 - acc: 0.5566 - val_loss: 0.6977 - val_acc: 0.5272

Epoch 00002: val_acc did not improve from 0.76693
Epoch 3/15
6020/6020 [==============================] - 3s 516us/step - loss: 0.6825 - acc: 0.5650 - val_loss: 0.6804 - val_acc: 0.5684

Epoch 00003: val_acc did not improve from 0.76693
Epoch 4/15
6020/6020 [==============================] - 3s 531us/step - loss: 0.6819 - acc: 0.5595 - val_loss: 0.6817 - val_acc: 0.5684

Epoch 00004: val_acc did not improve from 0.76693
Epoch 5/15
6020/6020 [==============================] - 3s 529us/step - loss: 0.6818 - acc: 0.5601 - val_loss: 0.6819 - val_acc: 0.5664

Epoch 00005: val_acc did not improve from 0.76693
Epoch 6/15
6020/602

In [86]:
from sklearn.metrics import confusion_matrix
y_pred = model_rnn.predict(X_test_rnn)
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

3202/3202 [==============================] - 5s 2ms/step
[[ 308 1331]
 [ 203 1360]]
              precision    recall  f1-score   support

           0       0.60      0.19      0.29      1639
           1       0.51      0.87      0.64      1563

    accuracy                           0.52      3202
   macro avg       0.55      0.53      0.46      3202
weighted avg       0.56      0.52      0.46      3202

